In [ ]:
import mlflow
from surprise import KNNWithMeans, SVD

from data_loader import DataLoader
from evaluation import Evaluation

In [ ]:
## optional set tracking uri to remote
# remote_server_uri = "http://127.0.0.1:5000/"  # set to your server URI
# mlflow.set_tracking_uri(remote_server_uri)

In [ ]:
# hyperparameter

n_epochs = 20
lr_all = 0.005

#### data_preparation

In [ ]:
trainset, testset = DataLoader().get_data()

#### setup experiment

In [ ]:
EXPERIMENT_NAME = "RecSys_MovieLens"

experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
if experiment is None:
    experiment_id = mlflow.create_experiment(EXPERIMENT_NAME) 
    experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)

## training and evaluation

In [ ]:
mlflow.start_run(experiment_id=experiment.experiment_id, run_name="SVD")
mlflow.log_param("n_epochs", n_epochs)
mlflow.log_param("lr_all", lr_all)

algo = SVD(n_epochs=n_epochs,lr_all=lr_all)

mlflow.set_tag("algorithm", type(algo))

algo.fit(trainset)

# evaluation
predictions = algo.test(testset)
results = Evaluation().evaluate(predictions)

mlflow.log_metrics(
metrics={
    f"RMSE": results['rmse'],
    f"Recall_10": results['recall@10'],
    f"Precision_10": results['precision@10'],
},)
    

#### Packaging

In [ ]:
# Define the model class
class PackedSVD(mlflow.pyfunc.PythonModel):
    def __init__(self, algo):
        self.n = algo

    def predict(self, context, model_input):
        return algo.predict( uid=model_input['uid'] ,  iid=model_input['iid'])
packed_model = PackedSVD(algo)

In [ ]:
mlflow.pyfunc.log_model(
    artifact_path="SVD",
    python_model=packed_model,
)

In [ ]:
mlflow.end_run()